# Capstone Project 3: Early Warning System - Combining Multiple Models

## Objective
This notebook walks you through the UPAD cycle (Understand, Prepare, Analyze, Deploy) to:
- Build an ensemble of models from different model families
- Implement a stacking classifier to combine model predictions
- Create risk scores and early warning tiers (High/Medium/Low risk)
- Develop student intervention recommendations
- Create deployment documentation for institutional use

# Understand

## Building a Production-Ready Early Warning System

Universities increasingly rely on early warning systems (EWS) to identify at-risk students before they leave. An effective EWS should:

1. **Combine multiple signals**: Different models may capture different patterns in student behavior
2. **Provide calibrated risk scores**: Not just binary predictions, but meaningful probability estimates
3. **Enable tiered interventions**: Different risk levels warrant different intervention intensities
4. **Be actionable**: Predictions should lead to specific intervention recommendations

### Ensemble Learning Approach

In this project, we will implement **stacking** (stacked generalization), which combines predictions from multiple base models using a meta-learner. The approach:

1. **Base Models**: Train diverse models (logistic regression, random forest, gradient boosting, neural network)
2. **Meta-Learner**: Train a second-level model on the base model predictions
3. **Final Prediction**: The meta-learner makes the final prediction

### Learning Objectives

By the end of this capstone, you will be able to:
1. Implement ensemble methods including stacking classifiers
2. Create calibrated risk scores and probability estimates
3. Design tiered intervention frameworks based on risk levels
4. Prepare deployment documentation for institutional use

# Prepare

## Data Wrangling

#### **Step 1: Import Libraries and Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

# Base Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Ensemble Methods
from sklearn.ensemble import StackingClassifier, VotingClassifier

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report, brier_score_loss
)

# Model persistence
import joblib
import json
from datetime import datetime

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("All libraries imported successfully!")

In [ ]:
# Load data
data_location = '/content/drive/MyDrive/projects/Applied-Data-Analytics-For-Higher-Education-Course-2/data/'
df = pd.read_csv(f'{data_location}student_academics_data.csv')
print(f"Dataset shape: {df.shape}")
df.head()

#### **Step 2: Data Cleaning**

In [ ]:
# Standard data cleaning
df['RACE_ETHNICITY'] = df['RACE_ETHNICITY'].replace(
    ['Unknown', 'Native Hawaiian or Other Pacific Islander', 'American Indian or Alaska Native'], 
    'Other'
)

df = df[df['GENDER'] != 'Nonbinary']
df['GENDER'] = df['GENDER'].str.strip().str.capitalize()

df.drop(['SEM_1_STATUS', 'SEM_2_STATUS'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

# Drop columns with >50% missing
missing_pct = df.isnull().sum() / len(df)
cols_to_drop = missing_pct[missing_pct > 0.5].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)

# Create target
df['DEPARTED'] = (df['SEM_3_STATUS'] != 'E').astype(int)

print(f"Cleaned dataset shape: {df.shape}")
print(f"Departure rate: {df['DEPARTED'].mean():.2%}")

#### **Step 3: Feature Engineering**

In [ ]:
# Feature engineering
def create_features(df):
    df = df.copy()
    
    # DFW Rates
    df['DFW_RATE_1'] = ((df['UNITS_ATTEMPTED_1'] - df['UNITS_COMPLETED_1']).clip(lower=0) 
                        / df['UNITS_ATTEMPTED_1'].replace(0, 1))
    df['DFW_RATE_2'] = ((df['UNITS_ATTEMPTED_2'] - df['UNITS_COMPLETED_2']).clip(lower=0) 
                        / df['UNITS_ATTEMPTED_2'].replace(0, 1))
    
    # Grade Points
    df['GRADE_POINTS_1'] = df['UNITS_ATTEMPTED_1'] * df['GPA_1']
    df['GRADE_POINTS_2'] = df['UNITS_ATTEMPTED_2'] * df['GPA_2']
    
    # GPA Trend (change from sem 1 to sem 2)
    df['GPA_TREND'] = df['GPA_2'] - df['GPA_1']
    
    # Cumulative metrics
    df['TOTAL_UNITS_ATTEMPTED'] = df['UNITS_ATTEMPTED_1'] + df['UNITS_ATTEMPTED_2']
    df['TOTAL_UNITS_COMPLETED'] = df['UNITS_COMPLETED_1'] + df['UNITS_COMPLETED_2']
    df['OVERALL_COMPLETION_RATE'] = df['TOTAL_UNITS_COMPLETED'] / df['TOTAL_UNITS_ATTEMPTED'].replace(0, 1)
    
    return df

df = create_features(df)
print("Features created successfully.")

#### **Step 4: Prepare Data for Modeling**

In [ ]:
# Define features
numeric_features = [
    'HS_GPA', 'HS_MATH_GPA', 'HS_ENGL_GPA',
    'UNITS_ATTEMPTED_1', 'UNITS_ATTEMPTED_2',
    'UNITS_COMPLETED_1', 'UNITS_COMPLETED_2',
    'DFW_UNITS_1', 'DFW_UNITS_2',
    'GPA_1', 'GPA_2',
    'DFW_RATE_1', 'DFW_RATE_2',
    'GRADE_POINTS_1', 'GRADE_POINTS_2',
    'GPA_TREND', 'OVERALL_COMPLETION_RATE'
]

categorical_features = ['RACE_ETHNICITY', 'GENDER', 'FIRST_GEN_STATUS', 'COLLEGE']

target = 'DEPARTED'

# Handle missing values
for col in numeric_features:
    if col in df.columns and df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

In [ ]:
# Train/Validation/Test split (60/20/20)
# First split off test set
train_val_df, test_df = train_test_split(
    df, test_size=0.2, random_state=RANDOM_STATE, stratify=df[target]
)

# Then split training into train and validation
train_df, val_df = train_test_split(
    train_val_df, test_size=0.25, random_state=RANDOM_STATE, stratify=train_val_df[target]
)

print(f"Training set: {len(train_df):,} students ({len(train_df)/len(df):.1%})")
print(f"Validation set: {len(val_df):,} students ({len(val_df)/len(df):.1%})")
print(f"Test set: {len(test_df):,} students ({len(test_df)/len(df):.1%})")

In [ ]:
# Prepare feature matrices
all_features = numeric_features + categorical_features
available_features = [f for f in all_features if f in train_df.columns]

train_encoded = pd.get_dummies(train_df[available_features], columns=categorical_features, drop_first=True)
val_encoded = pd.get_dummies(val_df[available_features], columns=categorical_features, drop_first=True)
test_encoded = pd.get_dummies(test_df[available_features], columns=categorical_features, drop_first=True)

# Align columns
train_encoded, val_encoded = train_encoded.align(val_encoded, join='left', axis=1, fill_value=0)
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

# Fill NaN
train_encoded = train_encoded.fillna(0)
val_encoded = val_encoded.fillna(0)
test_encoded = test_encoded.fillna(0)

# Prepare X and y
X_train = train_encoded
y_train = train_df[target]
X_val = val_encoded
y_val = val_df[target]
X_test = test_encoded
y_test = test_df[target]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Feature matrix shape: {X_train.shape}")

# Analyze

## Part 1: Build Base Models

#### **Step 5: Train Individual Base Models**

In [ ]:
# Define base models
base_models = {
    'Logistic Regression': LogisticRegression(
        penalty='l2', C=0.1, solver='lbfgs', max_iter=1000,
        class_weight='balanced', random_state=RANDOM_STATE
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=150, max_depth=10, min_samples_split=10,
        min_samples_leaf=5, class_weight='balanced',
        n_jobs=-1, random_state=RANDOM_STATE
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100, learning_rate=0.1, max_depth=5,
        min_samples_split=10, subsample=0.8,
        random_state=RANDOM_STATE
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(64, 32), activation='relu',
        solver='adam', alpha=0.01, max_iter=300,
        early_stopping=True, random_state=RANDOM_STATE
    )
}

# Models that need scaled data
scaled_models = ['Logistic Regression', 'Neural Network']

In [ ]:
# Train and evaluate each base model
base_results = []
trained_models = {}
base_predictions = {}
base_probabilities = {}

print("Training Base Models...")
print("="*80)

for name, model in base_models.items():
    print(f"\nTraining {name}...")
    
    # Select appropriate data
    if name in scaled_models:
        X_tr, X_v = X_train_scaled, X_val_scaled
    else:
        X_tr, X_v = X_train, X_val
    
    # Train
    model.fit(X_tr, y_train)
    trained_models[name] = model
    
    # Predict on validation
    y_pred = model.predict(X_v)
    y_prob = model.predict_proba(X_v)[:, 1]
    
    base_predictions[name] = y_pred
    base_probabilities[name] = y_prob
    
    # Evaluate
    results = {
        'Model': name,
        'Accuracy': accuracy_score(y_val, y_pred),
        'Precision': precision_score(y_val, y_pred, zero_division=0),
        'Recall': recall_score(y_val, y_pred, zero_division=0),
        'F1 Score': f1_score(y_val, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_val, y_prob),
        'Brier Score': brier_score_loss(y_val, y_prob)
    }
    base_results.append(results)
    
    print(f"  ROC-AUC: {results['ROC-AUC']:.4f}")
    print(f"  F1 Score: {results['F1 Score']:.4f}")

base_results_df = pd.DataFrame(base_results)
print("\n" + "="*80)
print("\nBase Model Results (Validation Set):")
print(base_results_df.round(4).to_string(index=False))

## Part 2: Build Ensemble Models

#### **Step 6: Implement Voting Classifier**

In [ ]:
# Create Voting Classifier (soft voting uses probability averages)
print("Building Voting Classifier...")

# For voting classifier, we need all models to use the same input
# We'll use scaled data and models that can handle it
voting_estimators = [
    ('lr', LogisticRegression(penalty='l2', C=0.1, max_iter=1000,
                              class_weight='balanced', random_state=RANDOM_STATE)),
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=10,
                                   class_weight='balanced', n_jobs=-1,
                                   random_state=RANDOM_STATE)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                       max_depth=5, random_state=RANDOM_STATE)),
    ('nn', MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300,
                         early_stopping=True, random_state=RANDOM_STATE))
]

voting_clf = VotingClassifier(
    estimators=voting_estimators,
    voting='soft',  # Use probability averaging
    n_jobs=-1
)

# Train
voting_clf.fit(X_train_scaled, y_train)

# Evaluate on validation
voting_pred = voting_clf.predict(X_val_scaled)
voting_prob = voting_clf.predict_proba(X_val_scaled)[:, 1]

print(f"\nVoting Classifier (Validation):")
print(f"  ROC-AUC: {roc_auc_score(y_val, voting_prob):.4f}")
print(f"  F1 Score: {f1_score(y_val, voting_pred):.4f}")
print(f"  Brier Score: {brier_score_loss(y_val, voting_prob):.4f}")

#### **Step 7: Implement Stacking Classifier**

In [ ]:
# Create Stacking Classifier
print("Building Stacking Classifier...")

# Base estimators for stacking
stacking_estimators = [
    ('lr', LogisticRegression(penalty='l2', C=0.1, max_iter=1000,
                              class_weight='balanced', random_state=RANDOM_STATE)),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10,
                                   class_weight='balanced', n_jobs=-1,
                                   random_state=RANDOM_STATE)),
    ('gb', GradientBoostingClassifier(n_estimators=80, learning_rate=0.1,
                                       max_depth=5, random_state=RANDOM_STATE))
]

# Meta-learner (final estimator)
meta_learner = LogisticRegression(
    penalty='l2', C=1.0, max_iter=1000, random_state=RANDOM_STATE
)

stacking_clf = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=meta_learner,
    cv=5,  # Cross-validation for generating meta-features
    stack_method='predict_proba',  # Use probabilities as meta-features
    n_jobs=-1
)

# Train
stacking_clf.fit(X_train_scaled, y_train)

# Evaluate on validation
stacking_pred = stacking_clf.predict(X_val_scaled)
stacking_prob = stacking_clf.predict_proba(X_val_scaled)[:, 1]

print(f"\nStacking Classifier (Validation):")
print(f"  ROC-AUC: {roc_auc_score(y_val, stacking_prob):.4f}")
print(f"  F1 Score: {f1_score(y_val, stacking_pred):.4f}")
print(f"  Brier Score: {brier_score_loss(y_val, stacking_prob):.4f}")

#### **Step 8: Compare All Models**

In [ ]:
# Add ensemble results to comparison
ensemble_results = [
    {
        'Model': 'Voting Ensemble',
        'Accuracy': accuracy_score(y_val, voting_pred),
        'Precision': precision_score(y_val, voting_pred, zero_division=0),
        'Recall': recall_score(y_val, voting_pred, zero_division=0),
        'F1 Score': f1_score(y_val, voting_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_val, voting_prob),
        'Brier Score': brier_score_loss(y_val, voting_prob)
    },
    {
        'Model': 'Stacking Ensemble',
        'Accuracy': accuracy_score(y_val, stacking_pred),
        'Precision': precision_score(y_val, stacking_pred, zero_division=0),
        'Recall': recall_score(y_val, stacking_pred, zero_division=0),
        'F1 Score': f1_score(y_val, stacking_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_val, stacking_prob),
        'Brier Score': brier_score_loss(y_val, stacking_prob)
    }
]

all_results_df = pd.concat([base_results_df, pd.DataFrame(ensemble_results)], ignore_index=True)
all_results_df = all_results_df.sort_values('ROC-AUC', ascending=False)

print("="*100)
print("COMPLETE MODEL COMPARISON (Validation Set)")
print("="*100)
print(all_results_df.round(4).to_string(index=False))
print("="*100)

In [ ]:
# Visualize model comparison
fig = go.Figure()

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']
colors = px.colors.qualitative.Set2

for i, metric in enumerate(metrics_to_plot):
    fig.add_trace(go.Bar(
        name=metric,
        x=all_results_df['Model'],
        y=all_results_df[metric],
        marker_color=colors[i % len(colors)]
    ))

fig.update_layout(
    title='Model Comparison: Base Models vs Ensembles',
    barmode='group',
    height=500,
    xaxis_tickangle=-30,
    legend=dict(orientation='h', y=1.1)
)
fig.show()

## Part 3: Create Risk Scores and Early Warning Tiers

#### **Step 9: Select Best Model and Calibrate Probabilities**

In [ ]:
# Select best model (using stacking ensemble as our production model)
best_model = stacking_clf
best_model_name = 'Stacking Ensemble'

print(f"Selected model for production: {best_model_name}")

In [ ]:
# Calibrate probabilities using isotonic regression
print("Calibrating probabilities...")

# We'll use CalibratedClassifierCV to improve probability estimates
calibrated_model = CalibratedClassifierCV(
    best_model, method='isotonic', cv='prefit'
)

# Fit calibration on validation set
calibrated_model.fit(X_val_scaled, y_val)

# Get calibrated probabilities on test set
calibrated_prob = calibrated_model.predict_proba(X_test_scaled)[:, 1]

# Compare calibration
uncalibrated_prob = best_model.predict_proba(X_test_scaled)[:, 1]

print(f"\nCalibration Results (Test Set):")
print(f"Uncalibrated Brier Score: {brier_score_loss(y_test, uncalibrated_prob):.4f}")
print(f"Calibrated Brier Score: {brier_score_loss(y_test, calibrated_prob):.4f}")

In [ ]:
# Visualize calibration curves
fig = make_subplots(rows=1, cols=2, subplot_titles=('Calibration Curve', 'Probability Distribution'))

# Calibration curve
fraction_positives_uncal, mean_predicted_uncal = calibration_curve(y_test, uncalibrated_prob, n_bins=10)
fraction_positives_cal, mean_predicted_cal = calibration_curve(y_test, calibrated_prob, n_bins=10)

fig.add_trace(
    go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Perfectly Calibrated',
               line=dict(dash='dash', color='gray')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=mean_predicted_uncal, y=fraction_positives_uncal, mode='lines+markers',
               name='Uncalibrated', line=dict(color='coral')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=mean_predicted_cal, y=fraction_positives_cal, mode='lines+markers',
               name='Calibrated', line=dict(color='steelblue')),
    row=1, col=1
)

# Probability distribution
fig.add_trace(
    go.Histogram(x=calibrated_prob, nbinsx=50, name='Risk Score Distribution',
                 marker_color='steelblue', opacity=0.7),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Model Calibration Analysis")
fig.update_xaxes(title_text="Mean Predicted Probability", row=1, col=1)
fig.update_yaxes(title_text="Fraction of Positives", row=1, col=1)
fig.update_xaxes(title_text="Risk Score", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

fig.show()

#### **Step 10: Define Risk Tiers**

In [ ]:
# Define risk tiers based on probability thresholds
def assign_risk_tier(probability):
    """
    Assign risk tier based on departure probability.
    Thresholds are set to balance intervention resources with identification accuracy.
    """
    if probability >= 0.5:
        return 'High Risk'
    elif probability >= 0.25:
        return 'Medium Risk'
    else:
        return 'Low Risk'

# Apply to test set
test_results = test_df.copy()
test_results['Risk_Score'] = calibrated_prob
test_results['Risk_Tier'] = [assign_risk_tier(p) for p in calibrated_prob]
test_results['Predicted_Departure'] = (calibrated_prob >= 0.5).astype(int)

# Summarize risk tier distribution
tier_summary = test_results.groupby('Risk_Tier').agg({
    'SID': 'count',
    'DEPARTED': ['sum', 'mean'],
    'Risk_Score': 'mean'
}).round(4)
tier_summary.columns = ['Count', 'Actual Departures', 'Departure Rate', 'Avg Risk Score']

print("="*80)
print("RISK TIER SUMMARY (Test Set)")
print("="*80)
print(tier_summary.to_string())
print("="*80)

In [ ]:
# Visualize risk tiers
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Students by Risk Tier', 'Departure Rate by Risk Tier'))

tier_counts = test_results['Risk_Tier'].value_counts()
tier_colors = {'High Risk': 'firebrick', 'Medium Risk': 'orange', 'Low Risk': 'forestgreen'}

# Count by tier
fig.add_trace(
    go.Bar(x=['High Risk', 'Medium Risk', 'Low Risk'],
           y=[tier_counts.get('High Risk', 0), tier_counts.get('Medium Risk', 0), tier_counts.get('Low Risk', 0)],
           marker_color=['firebrick', 'orange', 'forestgreen']),
    row=1, col=1
)

# Departure rate by tier
tier_rates = test_results.groupby('Risk_Tier')['DEPARTED'].mean()
fig.add_trace(
    go.Bar(x=['High Risk', 'Medium Risk', 'Low Risk'],
           y=[tier_rates.get('High Risk', 0), tier_rates.get('Medium Risk', 0), tier_rates.get('Low Risk', 0)],
           marker_color=['firebrick', 'orange', 'forestgreen'],
           text=[f"{tier_rates.get(t, 0):.1%}" for t in ['High Risk', 'Medium Risk', 'Low Risk']],
           textposition='outside'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Early Warning System: Risk Tier Analysis", showlegend=False)
fig.update_yaxes(title_text="Number of Students", row=1, col=1)
fig.update_yaxes(title_text="Actual Departure Rate", tickformat='.0%', row=1, col=2)

fig.show()

## Part 4: Develop Intervention Recommendations

#### **Step 11: Define Intervention Framework**

In [ ]:
# Define intervention recommendations based on risk tier and risk factors
intervention_framework = {
    'High Risk': {
        'Priority': 'Immediate',
        'Interventions': [
            'Mandatory meeting with academic advisor within 1 week',
            'Enrollment in academic success course',
            'Peer mentoring program assignment',
            'Weekly check-ins with success coach',
            'Financial aid review and emergency funding eligibility check'
        ],
        'Frequency': 'Weekly monitoring',
        'Escalation': 'Refer to Dean of Students if no engagement within 2 weeks'
    },
    'Medium Risk': {
        'Priority': 'Proactive',
        'Interventions': [
            'Outreach email with support resources',
            'Invitation to academic success workshop',
            'Optional tutoring center referral',
            'Study group matching',
            'Career counseling appointment offer'
        ],
        'Frequency': 'Bi-weekly monitoring',
        'Escalation': 'Upgrade to High Risk if GPA drops or engagement decreases'
    },
    'Low Risk': {
        'Priority': 'Standard',
        'Interventions': [
            'Automated wellness check-in survey',
            'General campus resource newsletter',
            'Leadership and involvement opportunities'
        ],
        'Frequency': 'Monthly monitoring',
        'Escalation': 'Upgrade if academic performance declines'
    }
}

# Display intervention framework
print("="*80)
print("INTERVENTION FRAMEWORK")
print("="*80)
for tier, details in intervention_framework.items():
    print(f"\n{tier.upper()}")
    print(f"Priority: {details['Priority']}")
    print(f"Monitoring Frequency: {details['Frequency']}")
    print(f"Interventions:")
    for intervention in details['Interventions']:
        print(f"  - {intervention}")
    print(f"Escalation: {details['Escalation']}")
print("\n" + "="*80)

#### **Step 12: Generate Student-Level Recommendations**

In [ ]:
def generate_student_recommendations(row, feature_importance=None):
    """
    Generate personalized intervention recommendations based on student profile.
    """
    recommendations = []
    risk_factors = []
    
    # Check academic performance
    if row['GPA_2'] < 2.0:
        risk_factors.append('Low GPA (below 2.0)')
        recommendations.append('Mandatory academic advising')
        recommendations.append('Tutoring center enrollment')
    
    if row['DFW_RATE_2'] > 0.3:
        risk_factors.append('High DFW rate')
        recommendations.append('Course load evaluation')
        recommendations.append('Study skills workshop')
    
    if row['GPA_TREND'] < -0.5:
        risk_factors.append('Declining GPA trend')
        recommendations.append('Academic recovery program')
    
    # Check for first-gen specific support
    if row['FIRST_GEN_STATUS'] == 'First Generation':
        recommendations.append('First-gen student support services referral')
    
    # Check unit completion
    if row['OVERALL_COMPLETION_RATE'] < 0.8:
        risk_factors.append('Low unit completion rate')
        recommendations.append('Withdrawal pattern review')
    
    return {
        'risk_factors': risk_factors if risk_factors else ['No specific risk factors identified'],
        'recommendations': recommendations if recommendations else ['Continue standard monitoring']
    }

In [ ]:
# Generate recommendations for high-risk students
high_risk_students = test_results[test_results['Risk_Tier'] == 'High Risk'].head(10)

print("="*80)
print("SAMPLE STUDENT INTERVENTION REPORTS (High Risk Students)")
print("="*80)

for idx, row in high_risk_students.iterrows():
    rec = generate_student_recommendations(row)
    print(f"\nStudent ID: {row['SID']}")
    print(f"Risk Score: {row['Risk_Score']:.2%}")
    print(f"Risk Tier: {row['Risk_Tier']}")
    print(f"Current GPA: {row['GPA_2']:.2f}")
    print(f"\nRisk Factors:")
    for factor in rec['risk_factors']:
        print(f"  - {factor}")
    print(f"\nRecommended Interventions:")
    for intervention in rec['recommendations']:
        print(f"  - {intervention}")
    print("-"*40)

# Deploy

## Part 5: Create Deployment Documentation

#### **Step 13: Final Model Evaluation on Test Set**

In [ ]:
# Final evaluation on test set
final_pred = (calibrated_prob >= 0.5).astype(int)

print("="*80)
print("FINAL MODEL EVALUATION (Test Set)")
print("="*80)
print(f"\nModel: {best_model_name} (Calibrated)")
print(f"Test Set Size: {len(y_test):,} students")
print(f"\nPerformance Metrics:")
print(f"  Accuracy: {accuracy_score(y_test, final_pred):.4f}")
print(f"  Precision: {precision_score(y_test, final_pred):.4f}")
print(f"  Recall: {recall_score(y_test, final_pred):.4f}")
print(f"  F1 Score: {f1_score(y_test, final_pred):.4f}")
print(f"  ROC-AUC: {roc_auc_score(y_test, calibrated_prob):.4f}")
print(f"  Brier Score: {brier_score_loss(y_test, calibrated_prob):.4f}")
print("="*80)

In [ ]:
# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, final_pred, target_names=['Enrolled', 'Departed']))

#### **Step 14: Create Deployment Documentation**

In [ ]:
# Generate deployment documentation
deployment_doc = {
    'model_info': {
        'name': 'Student Departure Early Warning System',
        'version': '1.0',
        'created_date': datetime.now().strftime('%Y-%m-%d'),
        'model_type': best_model_name,
        'calibration': 'Isotonic Regression'
    },
    'performance': {
        'test_set_size': len(y_test),
        'roc_auc': round(roc_auc_score(y_test, calibrated_prob), 4),
        'f1_score': round(f1_score(y_test, final_pred), 4),
        'precision': round(precision_score(y_test, final_pred), 4),
        'recall': round(recall_score(y_test, final_pred), 4),
        'brier_score': round(brier_score_loss(y_test, calibrated_prob), 4)
    },
    'features': {
        'numeric': numeric_features,
        'categorical': categorical_features,
        'total_features': X_train.shape[1]
    },
    'risk_tiers': {
        'high_risk': {'threshold': '>= 0.50', 'intervention': 'Immediate'},
        'medium_risk': {'threshold': '0.25 - 0.49', 'intervention': 'Proactive'},
        'low_risk': {'threshold': '< 0.25', 'intervention': 'Standard'}
    },
    'usage': {
        'input_format': 'CSV with required feature columns',
        'output_format': 'Risk score (0-1) and risk tier assignment',
        'refresh_frequency': 'Weekly recommended'
    },
    'limitations': [
        'Model trained on historical data; may not capture novel circumstances',
        'Requires complete feature data for accurate predictions',
        'Should be used as decision support, not automated decision-making',
        'Regular retraining recommended (annually at minimum)'
    ]
}

print("="*80)
print("DEPLOYMENT DOCUMENTATION")
print("="*80)
print(json.dumps(deployment_doc, indent=2))

#### **Step 15: Produce Comprehensive Deployment Report**

### Deliverable: Early Warning System Deployment Documentation

Using the analyses above, write a comprehensive deployment report that addresses the following:

1. **System Overview**: Describe the early warning system, including:
   - Purpose and goals
   - Model architecture (ensemble approach)
   - Key performance metrics

2. **Technical Specifications**: Document the technical details:
   - Required input features and data formats
   - Model components (base models, meta-learner)
   - Calibration methodology
   - Output interpretation

3. **Risk Tier Framework**: Explain the intervention tiers:
   - Threshold definitions and rationale
   - Expected student distribution across tiers
   - Validation of tier effectiveness

4. **Intervention Recommendations**: Provide guidance on:
   - Interventions for each risk tier
   - Personalization based on risk factors
   - Escalation procedures
   - Success metrics for interventions

5. **Operational Considerations**: Address implementation needs:
   - Data pipeline requirements
   - Refresh/update schedule
   - Monitoring and alerting
   - Staff training needs

6. **Limitations and Ethical Guidelines**: Discuss:
   - Known limitations of the model
   - Appropriate and inappropriate uses
   - Privacy considerations
   - Bias monitoring and mitigation

> **Rubric**: Your report should be 4-5 pages and include:
> - Complete technical documentation
> - Risk tier visualization and validation
> - Intervention framework with specific recommendations
> - Operational deployment plan
> - Ethical guidelines and limitations

---

## Your Report (Write Below)

*[Write your comprehensive deployment documentation here]*

---